In [23]:
import pandas as pd
import sqlite3
import backtrader as bt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import Dash, dcc, html, Input, Output, State, clientside_callback, no_update
import dash_bootstrap_components as dbc
import numpy as np
from datetime import datetime
from io import StringIO

# 导入策略
try:
    from strategies.kama_strategy import KAMAStrategy
    from strategies.rsi_strategy import RSIStrategy
    print("成功导入 KAMAStrategy 和 RSIStrategy")
except ImportError as e:
    print(f"导入失败：{e}")
    raise

# 数据加载函数
def load_kline_data(db_path, coin_name='LEAUSDT', interval='1m', limit=600, offset=0):
    try:
        conn = sqlite3.connect(db_path)
        query = f"""
            SELECT timestamp, open, high, low, close, volume
            FROM kline_data
            WHERE coin_name = ? AND interval = ?
            ORDER BY timestamp DESC
            LIMIT ? OFFSET ?
        """
        df = pd.read_sql_query(query, conn, params=(coin_name, interval, limit, offset))
        conn.close()
        
        if df.empty:
            print("警告：数据库返回空数据")
            return pd.DataFrame()
        
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
        df.set_index('timestamp', inplace=True)
        df = df.sort_index()
        df.columns = ['open', 'high', 'low', 'close', 'volume']
        
        if df[['open', 'high', 'low', 'close']].le(0).any().any() or df.isna().any().any():
            print("警告：数据包含零、负值或 NaN")
            df = df[(df[['open', 'high', 'low', 'close']] > 0).all(axis=1)].dropna()
        
        print(f"加载数据：{len(df)} 条，时间范围：{df.index.min()} 到 {df.index.max()}")
        print(f"价格范围：open {df['open'].min()} - {df['open'].max()}, close {df['close'].min()} - {df['close'].max()}")
        
        return df
    except Exception as e:
        print(f"数据加载错误：{e}")
        return pd.DataFrame()

# Backtrader 数据馈送
class PandasData(bt.feeds.PandasData):
    params = (
        ('datetime', None),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', None),
    )

# 回测函数
def run_backtest(data, strategy_class, **strategy_params):
    if data.empty or len(data) < max([v for k, v in strategy_params.items() if 'period' in k.lower()] or [10]):
        print("错误：数据不足，无法回测")
        return None, pd.DataFrame(), pd.DataFrame()
    
    cerebro = bt.Cerebro()
    cerebro.addstrategy(strategy_class, **strategy_params)
    
    data_feed = PandasData(dataname=data)
    cerebro.adddata(data_feed)
    
    cerebro.broker.setcash(1000000)
    cerebro.broker.setcommission(commission=0.001)
    
    try:
        cerebro.run()
        strategy = cerebro.runstrats[0][0]
        trades = strategy.trades
        
        if trades:
            trades_df = pd.DataFrame(trades)
            buy_signals = trades_df[trades_df['Size'] > 0][['EntryTime', 'EntryPrice']]
            sell_signals = trades_df[trades_df['Size'] < 0][['EntryTime', 'EntryPrice']]
        else:
            buy_signals = pd.DataFrame(columns=['EntryTime', 'EntryPrice'])
            sell_signals = pd.DataFrame(columns=['EntryTime', 'EntryPrice'])
        
        final_value = cerebro.broker.getvalue()
        initial_value = 1000000
        returns = (final_value - initial_value) / initial_value * 100
        stats = {
            'Return [%]': returns,
            'Max. Drawdown [%]': 0,
            '# Trades': len(trades),
            'Win Rate [%]': 0,
        }
        
        print(f"回测完成：交易次数 {len(trades)}")
        return stats, buy_signals, sell_signals
    except Exception as e:
        print(f"回测错误：{e}")
        return None, pd.DataFrame(), pd.DataFrame()

# 绘图函数
def create_kline_plot(data, buy_signals, sell_signals, strategy_type='kama', indicator_values=None, indicator_name='Indicator'):
    if data.empty:
        print("警告：无数据，无法绘图")
        return go.Figure()
    
    if strategy_type == 'rsi':
        fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                            vertical_spacing=0.03, subplot_titles=('K线图', '成交量', 'RSI'),
                            row_heights=[0.5, 0.2, 0.3])
    else:
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                            vertical_spacing=0.03, subplot_titles=('K线图', '成交量'),
                            row_heights=[0.7, 0.3])
    
    fig.add_trace(
        go.Candlestick(
            x=data.index,
            open=data['open'], high=data['high'],
            low=data['low'], close=data['close'],
            name='K线'
        ),
        row=1, col=1
    )
    
    if indicator_values is not None:
        if strategy_type == 'kama':
            fig.add_trace(
                go.Scatter(x=data.index, y=indicator_values, name=indicator_name, line=dict(color='orange')),
                row=1, col=1
            )
        elif strategy_type == 'rsi':
            fig.add_trace(
                go.Scatter(x=data.index, y=indicator_values, name='RSI', line=dict(color='purple')),
                row=3, col=1
            )
            fig.add_shape(
                type='line', x0=data.index.min(), x1=data.index.max(),
                y0=70, y1=70, line=dict(color='red', dash='dash'),
                row=3, col=1
            )
            fig.add_shape(
                type='line', x0=data.index.min(), x1=data.index.max(),
                y0=30, y1=30, line=dict(color='green', dash='dash'),
                row=3, col=1
            )
    
    if not buy_signals.empty:
        fig.add_trace(
            go.Scatter(
                x=buy_signals['EntryTime'], y=buy_signals['EntryPrice'],
                mode='markers', name='买入',
                marker=dict(symbol='triangle-up', size=10, color='green')
            ),
            row=1, col=1
        )
    if not sell_signals.empty:
        fig.add_trace(
            go.Scatter(
                x=sell_signals['EntryTime'], y=sell_signals['EntryPrice'],
                mode='markers', name='卖出',
                marker=dict(symbol='triangle-down', size=10, color='red')
            ),
            row=1, col=1
        )
    
    fig.add_trace(
        go.Bar(x=data.index, y=data['volume'], name='成交量', marker_color='blue'),
        row=2, col=1
    )
    
    if len(data) >= 200:
        end_date = data.index.max()
        start_date = data.index[-200]
        xaxis_range = [start_date, end_date]
    else:
        xaxis_range = [data.index.min(), data.index.max()]
    
    fig.update_layout(
        xaxis_rangeslider_visible=False,
        dragmode='pan',
        template='plotly_dark',
        hovermode='x unified',
        showlegend=True,
        height=800 if strategy_type == 'rsi' else 600,
        autosize=True,
        xaxis=dict(range=xaxis_range)
    )
    fig.update_xaxes(
        rangebreaks=[dict(bounds=['sat', 'mon'])]
    )
    
    return fig

# Dash 应用
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY], suppress_callback_exceptions=True)

# 数据库路径
DB_PATH = r'D:\策略研究\kline_db_new\kline_data_LEAUSDT.db'

# 初始数据
initial_data = load_kline_data(DB_PATH, limit=600)

# 计算 KAMA
def calculate_kama(series, period=3, fast_period=2, slow_period=30):
    kama = np.zeros(len(series))
    kama[0] = series[0]
    fast = 2 / (fast_period + 1)
    slow = 2 / (slow_period + 1)
    
    for i in range(1, len(series)):
        if i < period:
            kama[i] = series[i]
            continue
        change = abs(series[i] - series[i - period])
        volatility = sum(abs(series[j] - series[j-1]) for j in range(i - period + 1, i + 1))
        er = change / volatility if volatility != 0 else 0
        sc = (er * (fast - slow) + slow) ** 2
        kama[i] = kama[i-1] + sc * (series[i] - kama[i-1])
    
    return kama

# 计算 RSI
def calculate_rsi(series, period=14):
    delta = np.diff(series)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    
    avg_gain = np.zeros(len(series))
    avg_loss = np.zeros(len(series))
    avg_gain[period] = np.mean(gain[:period])
    avg_loss[period] = np.mean(loss[:period])
    
    for i in range(period + 1, len(gain)):
        avg_gain[i] = (avg_gain[i-1] * (period - 1) + gain[i-1]) / period
        avg_loss[i] = (avg_loss[i-1] * (period - 1) + loss[i-1]) / period
    
    rs = np.divide(avg_gain, avg_loss, out=np.zeros_like(avg_gain), where=avg_loss != 0)
    rsi = 100 - (100 / (1 + rs))
    rsi[:period] = np.nan
    return rsi

# 初始回测（默认 KAMA）
initial_stats, initial_buy_signals, initial_sell_signals = run_backtest(
    initial_data, KAMAStrategy, kama_period=3, fast_period=2, slow_period=30
)

# 初始 KAMA 指标
initial_kama = calculate_kama(initial_data['close'].values, period=3)
print(f"KAMA 范围：{initial_kama.min():.6f} - {initial_kama.max():.6f}")

# 初始 RSI 指标
initial_rsi = calculate_rsi(initial_data['close'].values, period=14)
print(f"RSI 范围：{np.nanmin(initial_rsi):.2f} - {np.nanmax(initial_rsi):.2f}")

# 初始图表
initial_fig = create_kline_plot(
    initial_data, initial_buy_signals, initial_sell_signals,
    strategy_type='kama', indicator_values=initial_kama, indicator_name='KAMA3'
)

# 布局
app.layout = dbc.Container([
    html.H1('LEAUSDT 策略回测', style={'textAlign': 'center'}),
    dbc.Row([
        dbc.Col([
            html.Label('选择策略'),
            dcc.Dropdown(
                id='strategy-select',
                options=[
                    {'label': 'KAMA', 'value': 'kama'},
                    {'label': 'RSI', 'value': 'rsi'},
                ],
                value='kama'
            ),
            html.Div(id='strategy-params'),
            html.Br(),
            dbc.Button('更新回测', id='update-btn', color='primary', n_clicks=0),
            html.Br(),
            html.Label('加载历史数据'),
            dbc.Button('加载更多', id='load-more', color='secondary', n_clicks=0),
        ], width=3),
        dbc.Col([
            dcc.Graph(id='kline-plot', figure=initial_fig, config={'scrollZoom': True}),
            html.Div(id='stats-output')
        ], width=9)
    ]),
    dcc.Store(id='data-store', data=initial_data.to_json(date_format='iso', orient='split')),
    dcc.Store(id='offset-store', data=0),
    dcc.Store(id='kama-params', data={'kama_period': 3, 'fast_period': 2, 'slow_period': 30}),
    dcc.Store(id='rsi-params', data={'rsi_period': 14, 'overbought': 70, 'oversold': 30}),
], fluid=True)

# 动态参数输入框
@app.callback(
    Output('strategy-params', 'children'),
    Input('strategy-select', 'value'),
    State('kama-params', 'data'),
    State('rsi-params', 'data')
)
def update_params(strategy, kama_params, rsi_params):
    kama_params = kama_params or {'kama_period': 3, 'fast_period': 2, 'slow_period': 30}
    rsi_params = rsi_params or {'rsi_period': 14, 'overbought': 70, 'oversold': 30}
    
    if strategy == 'kama':
        return [
            html.Label('KAMA 周期'),
            dcc.Input(id='kama-period', type='number', value=kama_params['kama_period'], min=1, step=1),
            html.Label('快周期'),
            dcc.Input(id='fast-period', type='number', value=kama_params['fast_period'], min=1, step=1),
            html.Label('慢周期'),
            dcc.Input(id='slow-period', type='number', value=kama_params['slow_period'], min=1, step=1),
        ]
    elif strategy == 'rsi':
        return [
            html.Label('RSI 周期'),
            dcc.Input(id='rsi-period', type='number', value=rsi_params['rsi_period'], min=1, step=1),
            html.Label('超买阈值'),
            dcc.Input(id='overbought', type='number', value=rsi_params['overbought'], min=0, max=100, step=1),
            html.Label('超卖阈值'),
            dcc.Input(id='oversold', type='number', value=rsi_params['oversold'], min=0, max=100, step=1),
        ]
    return []

# 客户端回调：保存 KAMA 参数
clientside_callback(
    """
    function(kama_period, fast_period, slow_period) {
        return {
            'kama_period': kama_period || 3,
            'fast_period': fast_period || 2,
            'slow_period': slow_period || 30
        };
    }
    """,
    Output('kama-params', 'data'),
    [Input('kama-period', 'value'),
     Input('fast-period', 'value'),
     Input('slow-period', 'value')]
)

# 客户端回调：保存 RSI 参数
clientside_callback(
    """
    function(rsi_period, overbought, oversold) {
        return {
            'rsi_period': rsi_period || 14,
            'overbought': overbought || 70,
            'oversold': oversold || 30
        };
    }
    """,
    Output('rsi-params', 'data'),
    [Input('rsi-period', 'value'),
     Input('overbought', 'value'),
     Input('oversold', 'value')]
)

# 更新图表和统计
@app.callback(
    [Output('kline-plot', 'figure'),
     Output('stats-output', 'children'),
     Output('data-store', 'data'),
     Output('offset-store', 'data')],
    [Input('update-btn', 'n_clicks'),
     Input('load-more', 'n_clicks')],
    [State('strategy-select', 'value'),
     State('kama-params', 'data'),
     State('rsi-params', 'data'),
     State('data-store', 'data'),
     State('offset-store', 'data')]
)
def update_plot(update_clicks, load_clicks, strategy, kama_params, rsi_params, stored_data, offset):
    kama_params = kama_params or {'kama_period': 3, 'fast_period': 2, 'slow_period': 30}
    rsi_params = rsi_params or {'rsi_period': 14, 'overbought': 70, 'oversold': 30}
    
    data = pd.read_json(StringIO(stored_data), orient='split')
    data.index = pd.to_datetime(data.index)
    
    triggered_id = dash.callback_context.triggered[0]['prop_id'].split('.')[0]
    if triggered_id == 'load-more' and load_clicks > 0:
        new_offset = offset + 100
        new_data = load_kline_data(DB_PATH, limit=100, offset=new_offset)
        if not new_data.empty:
            data = pd.concat([new_data, data]).sort_index()
    else:
        new_offset = offset
    
    if strategy == 'kama':
        stats, buy_signals, sell_signals = run_backtest(
            data, KAMAStrategy,
            kama_period=kama_params['kama_period'],
            fast_period=kama_params['fast_period'],
            slow_period=kama_params['slow_period']
        )
        indicator_values = calculate_kama(data['close'].values,
                                         period=kama_params['kama_period'],
                                         fast_period=kama_params['fast_period'],
                                         slow_period=kama_params['slow_period'])
        fig = create_kline_plot(
            data, buy_signals, sell_signals,
            strategy_type='kama',
            indicator_values=indicator_values,
            indicator_name=f'KAMA{kama_params["kama_period"]}'
        )
    else:  # rsi
        stats, buy_signals, sell_signals = run_backtest(
            data, RSIStrategy,
            rsi_period=rsi_params['rsi_period'],
            overbought=rsi_params['overbought'],
            oversold=rsi_params['oversold']
        )
        indicator_values = calculate_rsi(data['close'].values, period=rsi_params['rsi_period'])
        fig = create_kline_plot(
            data, buy_signals, sell_signals,
            strategy_type='rsi',
            indicator_values=indicator_values,
            indicator_name='RSI'
        )
    
    if stats is None:
        stats_text = [html.P("无足够数据进行回测")]
    else:
        stats_text = [
            html.P(f"总回报: {stats['Return [%]']:.2f}%"),
            html.P(f"最大回撤: {stats['Max. Drawdown [%]']:.2f}%"),
            html.P(f"交易次数: {stats['# Trades']}"),
            html.P(f"胜率: {stats['Win Rate [%]']:.2f}%")
        ]
    
    data_json = data.to_json(date_format='iso', orient='split')
    return fig, stats_text, data_json, new_offset

# 滚动加载
@app.callback(
    [Output('kline-plot', 'figure', allow_duplicate=True),
     Output('data-store', 'data', allow_duplicate=True),
     Output('offset-store', 'data', allow_duplicate=True)],
    Input('kline-plot', 'relayoutData'),
    [State('data-store', 'data'),
     State('offset-store', 'data'),
     State('strategy-select', 'value'),
     State('kama-params', 'data'),
     State('rsi-params', 'data')],
    prevent_initial_call=True
)
def load_on_scroll(relayout_data, stored_data, offset, strategy, kama_params, rsi_params):
    kama_params = kama_params or {'kama_period': 3, 'fast_period': 2, 'slow_period': 30}
    rsi_params = rsi_params or {'rsi_period': 14, 'overbought': 70, 'oversold': 30}
    
    if relayout_data and 'xaxis.range[0]' in relayout_data:
        data = pd.read_json(StringIO(stored_data), orient='split')
        data.index = pd.to_datetime(data.index)
        start_date = pd.to_datetime(relayout_data['xaxis.range[0]'])
        
        if start_date <= data.index[:100].max():
            new_offset = offset + 100
            new_data = load_kline_data(DB_PATH, limit=100, offset=new_offset)
            if not new_data.empty:
                data = pd.concat([new_data, data]).sort_index()
                if strategy == 'kama':
                    stats, buy_signals, sell_signals = run_backtest(
                        data, KAMAStrategy,
                        kama_period=kama_params['kama_period'],
                        fast_period=kama_params['fast_period'],
                        slow_period=kama_params['slow_period']
                    )
                    indicator_values = calculate_kama(data['close'].values,
                                                     period=kama_params['kama_period'],
                                                     fast_period=kama_params['fast_period'],
                                                     slow_period=kama_params['slow_period'])
                    fig = create_kline_plot(
                        data, buy_signals, sell_signals,
                        strategy_type='kama',
                        indicator_values=indicator_values,
                        indicator_name=f'KAMA{kama_params["kama_period"]}'
                    )
                else:  # rsi
                    stats, buy_signals, sell_signals = run_backtest(
                        data, RSIStrategy,
                        rsi_period=rsi_params['rsi_period'],
                        overbought=rsi_params['overbought'],
                        oversold=rsi_params['oversold']
                    )
                    indicator_values = calculate_rsi(data['close'].values, period=rsi_params['rsi_period'])
                    fig = create_kline_plot(
                        data, buy_signals, sell_signals,
                        strategy_type='rsi',
                        indicator_values=indicator_values,
                        indicator_name='RSI'
                    )
                data_json = data.to_json(date_format='iso', orient='split')
                return fig, data_json, new_offset
    return no_update, no_update, no_update

# 运行应用
if __name__ == '__main__':
    app.run(debug=True)

成功导入 KAMAStrategy 和 RSIStrategy
加载数据：600 条，时间范围：2025-04-16 18:44:00 到 2025-04-17 04:43:00
价格范围：open 0.00298764 - 0.00452598, close 0.00298764 - 0.00452598
执行 next：时间 2025-04-16 18:46:00
检查交叉：时间 2025-04-16 18:46:00, Close 0.003406, KAMA nan, Prev Close 0.003389, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 18:47:00
检查交叉：时间 2025-04-16 18:47:00, Close 0.003365, KAMA nan, Prev Close 0.003406, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 18:48:00
检查交叉：时间 2025-04-16 18:48:00, Close 0.003442, KAMA nan, Prev Close 0.003365, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 18:49:00
检查交叉：时间 2025-04-16 18:49:00, Close 0.003447, KAMA nan, Prev Close 0.003442, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 18:50:00
检查交叉：时间 2025-04-16 18:50:00, Close 0.003509, KAMA nan, Prev Close 0.003447, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 18:51:00
检查交叉：时间 2025-04-16 18:51:00, Close 0.003423, KAMA nan, Prev Close 0.003509, Prev KAMA nan
买入条件：Fals

执行 next：时间 2025-04-16 18:46:00
检查交叉：时间 2025-04-16 18:46:00, Close 0.003406, KAMA nan, Prev Close 0.003389, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 18:47:00
检查交叉：时间 2025-04-16 18:47:00, Close 0.003365, KAMA nan, Prev Close 0.003406, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 18:48:00
检查交叉：时间 2025-04-16 18:48:00, Close 0.003442, KAMA nan, Prev Close 0.003365, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 18:49:00
检查交叉：时间 2025-04-16 18:49:00, Close 0.003447, KAMA nan, Prev Close 0.003442, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 18:50:00
检查交叉：时间 2025-04-16 18:50:00, Close 0.003509, KAMA nan, Prev Close 0.003447, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 18:51:00
检查交叉：时间 2025-04-16 18:51:00, Close 0.003423, KAMA nan, Prev Close 0.003509, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 18:52:00
检查交叉：时间 2025-04-16 18:52:00, Close 0.003339, KAMA nan, Prev Close 0.003423, Prev KAMA nan
买入条件：False
卖出条件：False

成功导入 KAMAStrategy 和 RSIStrategy
加载数据：600 条，时间范围：2025-04-16 05:18:00 到 2025-04-16 15:17:00
价格范围：open 0.00316723 - 0.00582175, close 0.00316723 - 0.00582175
执行 next：时间 2025-04-16 05:20:00
检查交叉：时间 2025-04-16 05:20:00, Close 0.004310, KAMA nan, Prev Close 0.004365, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 05:21:00
检查交叉：时间 2025-04-16 05:21:00, Close 0.004403, KAMA nan, Prev Close 0.004310, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 05:22:00
检查交叉：时间 2025-04-16 05:22:00, Close 0.004378, KAMA nan, Prev Close 0.004403, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 05:23:00
检查交叉：时间 2025-04-16 05:23:00, Close 0.004406, KAMA nan, Prev Close 0.004378, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 05:24:00
检查交叉：时间 2025-04-16 05:24:00, Close 0.004498, KAMA nan, Prev Close 0.004406, Prev KAMA nan
买入条件：False
卖出条件：False
执行 next：时间 2025-04-16 05:25:00
检查交叉：时间 2025-04-16 05:25:00, Close 0.004533, KAMA nan, Prev Close 0.004498, Prev KAMA nan
买入条件：Fals